# Task4

In [37]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')



# Matrix and Decomposition

In [104]:
M= np.matrix('1 0 0 0 ;0 0 -1 0;0 -1 0 0;0 0 0 1')

In [105]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [106]:
print(H)

(0.5) [I0 I1]
+ (-0.5) [X0 X1]
+ (-0.5) [Y0 Y1]
+ (0.5) [Z0 Z1]


# Ansatz

In [97]:
def Operational_circuit(params):
    qc = QuantumCircuit(2)
    
    #qc.h(range(2))
    qc.rx(params[0],0)
    qc.rx(params[1],1)
    qc.ry(params[2],0)
    qc.ry(params[3],1)
    qc.cx(0,1)
    qc.ry(params[4],0)
    qc.ry(params[5],1)
    
    return qc
    

In [98]:
Operational_circuit_try = Operational_circuit([1,2,3,4,5,6])

In [103]:
Operational_circuit_try.draw()

┌───────┐┌───────┐     ┌───────┐
q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├
     ├───────┤├───────┤┌─┴─┐├───────┤
q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├
     └───────┘└───────┘└───┘└───────┘

# Measurments for different components of H

# 1. ZZ

In [68]:
def measure_zz_circuit(given_circuit):
    zz_meas = given_circuit.copy()
    zz_meas.measure_all()
    return zz_meas

zz_meas = measure_zz_circuit(Operational_circuit_try)
zz_meas.draw()

┌───────┐┌───────┐     ┌───────┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(4) ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(3) ├┤ X ├┤ RY(4) ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘ ░  ║ └╥┘
meas_0: ════════════════════════════════════╩══╬═
                                               ║ 
meas_1: ═══════════════════════════════════════╩═

In [34]:
def measure_zz(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

In [69]:
zz = measure_zz(Operational_circuit_try)
print("<ZZ> =", str(zz))

<ZZ> = 0.5868


# 2. XX

In [39]:
def measure_xx_circuit(given_circuit):
    xx_meas = given_circuit.copy()
    xx_meas.h(0)
    xx_meas.h(1)
    xx_meas.measure_all()

    return xx_meas

In [70]:
xx_meas = measure_xx_circuit(Operational_circuit_try)
xx_meas.draw()

┌───────┐┌───────┐     ┌───────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(4) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(3) ├┤ X ├┤ RY(4) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ═════════════════════════════════════════╩══╬═
                                                    ║ 
meas_1: ════════════════════════════════════════════╩═

In [41]:
def measure_xx(given_circuit, num_shots = 10000):
    
    xx_meas = measure_xx_circuit(given_circuit)
    
    result = execute(xx_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xx_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xx = counts['00'] + counts['11'] - counts['01'] - counts['10']
    xx = xx / total_counts
    
    return xx

In [71]:
xx = measure_xx(Operational_circuit_try)
print("<XX> =", str(xx))

<XX> = -0.1034


# 3. YY

In [108]:
def measure_yy_circuit(given_circuit):
    yy_meas = given_circuit.copy()
    yy_meas.rz(-pi/2,0)
    yy_meas.rz(-pi/2,1)

    yy_meas.h(0)
    yy_meas.h(1)
    yy_meas.measure_all()

    return yy_meas

In [109]:
yy_meas = measure_yy_circuit(Operational_circuit_try)
yy_meas.draw()

┌───────┐┌───────┐     ┌───────┐┌───────────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤├───────────┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘└───────────┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

In [110]:
def measure_yy(given_circuit, num_shots = 10000):
    
    yy_meas = measure_yy_circuit(given_circuit)
    
    result = execute(yy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yy = counts['00'] + counts['11'] - counts['01'] - counts['10']
    yy = yy / total_counts
    
    return yy

In [111]:
yy = measure_yy(Operational_circuit_try)
print("<yy> =", str(yy))

<yy> = -0.0234


# Cost_Function

In [154]:
def eigen_value(params):
    num_shots = 10000
    zz = measure_zz(Operational_circuit(params), num_shots = num_shots)
    xx = measure_xx(Operational_circuit(params), num_shots = num_shots)
    yy = measure_yy(Operational_circuit(params), num_shots = num_shots)
    
    ev = (0.5)*1 + (0.5)*zz + (-0.5)*xx+(-0.5)*yy
    
    return ev

In [155]:
params = [0,0,0,0,0,0]

In [156]:
ev = eigen_value(params)     #Try energy with params = [0,0,0,0,0,0]
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is 1.0070999999999999


# Finding Lowest Eignevalue

In [157]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(50,0.001)

In [158]:
ret = optimizer.optimize(num_vars=6, objective_function=eigen_value, initial_point=params)

In [159]:
eigen_value(ret[0])

-0.9931